--- Day 12: Digital Plumber ---  
Walking along the memory banks of the stream, you find a small village that is experiencing a little confusion:  
some programs can't communicate with each other.  

Programs in this village communicate using a fixed system of pipes.  
Messages are passed between programs using these pipes, but most programs aren't connected to each other directly.   Instead, programs pass messages between each other until the message reaches the intended recipient.  

For some reason, though, some of these messages aren't ever reaching their intended recipient,  
and the programs suspect that some pipes are missing. They would like you to investigate.  

You walk through the village and record the ID of each program  
and the IDs with which it can communicate directly (your puzzle input).  
Each program has one or more programs with which it can communicate,  
and these pipes are bidirectional;  
if 8 says it can communicate with 11, then 11 will say it can communicate with 8.  

You need to figure out how many programs are in the group that contains program ID 0.  

For example, suppose you go door-to-door like a travelling salesman and record the following list:  

0 <-> 2  
1 <-> 1  
2 <-> 0, 3, 4  
3 <-> 2, 4   
4 <-> 2, 3, 6  
5 <-> 6  
6 <-> 4, 5  

In this example, the following programs are in the group that contains program ID 0:  

Program 0 by definition.  
Program 2, directly connected to program 0.  
Program 3 via program 2.  
Program 4 via program 2.  
Program 5 via programs 6, then 4, then 2.  
Program 6 via programs 4, then 2.  
Therefore, a total of 6 programs are in this group;  
all but program 1, which has a pipe that connects it to itself.  

How many programs are in the group that contains program ID 0?  

In [1]:
dictionary={
    0:[2],
    1:[1],
    2:[0,3,4],
    3:[2,4],
    4:[2,3,6],
    5:[6],
    6:[4,5]
}

In [2]:
# back tracking algorithm modified from https://www.python.org/doc/essays/graphs/

def check_connected(dictionary,start,end,path=[]):
    """Return path if start and end are connected, else return None"""
    path=path+[start]
    if start==end:
        return path
    for node in dictionary[start]:
        if node not in path:
            newpath=check_connected(dictionary,node,end,path)
            if newpath: return newpath
    return None

In [3]:
# original algo
def find_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if not graph.has_key(start):
            return None
        for node in graph[start]:
            if node not in path:
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [4]:
check_connected(dictionary,2,0)

[2, 0]

In [5]:
check_connected(dictionary,0,2)

[0, 2]

In [6]:
check_connected(dictionary,0,3)

[0, 2, 3]

In [7]:
check_connected(dictionary,0,6)

[0, 2, 3, 4, 6]

In [8]:
def count_nodes(dictionary, target,counter=0):
    """Count the number of nodes that connect to target node"""
    for key, value in dictionary.iteritems():
        if check_connected(dictionary,key,target):
            counter+=1
    return counter

In [9]:
count_nodes(dictionary,0)

6

In [10]:
with open('inputs.txt','r') as f:
    inputs=f.readlines()
    inputs=[i.strip('\n') for i in inputs]
    keys=[int(i.split(' <')[0]) for i in inputs]
    values=[i.split('> ')[1] for i in inputs]
    values=[v.split(', ') for v in values]
print inputs[:5]
print keys[:5]
print values[:5]

['0 <-> 46, 1376', '1 <-> 1465, 1889', '2 <-> 609, 1578', '3 <-> 3, 1003, 1133, 1887', '4 <-> 467, 1282']
[0, 1, 2, 3, 4]
[['46', '1376'], ['1465', '1889'], ['609', '1578'], ['3', '1003', '1133', '1887'], ['467', '1282']]


In [11]:
# I can't find a more pythonic way to parse int from a comma separated string
values2=[]
for v in values:
    entry=[int(a) for a in v]
    values2.append(entry)
print values2[:5]

[[46, 1376], [1465, 1889], [609, 1578], [3, 1003, 1133, 1887], [467, 1282]]


In [12]:
# create new dictionary
dictionary={}
for k,v in zip(keys,values2):
    dictionary[k]=v

In [13]:
dict(dictionary.items()[0:5]) #hack to view head of dictionary

{0: [46, 1376],
 1: [1465, 1889],
 2: [609, 1578],
 3: [3, 1003, 1133, 1887],
 4: [467, 1282]}

In [14]:
count_nodes(dictionary,0)

152

--- Part Two ---  
There are more programs than just the ones in the group containing program ID 0.  
The rest of them have no way of reaching that group, and still might have no way of reaching each other.  

A group is a collection of programs that can all communicate via pipes either directly or indirectly.  
The programs you identified just a moment ago are all part of the same group.  
Now, they would like you to determine the total number of groups.  

In the example above, there were 2 groups: one consisting of programs 0,2,3,4,5,6,  
and the other consisting solely of program 1.  

How many groups are there in total?  

In [15]:
def count_nodes2(dictionary, target):
    """Count the number of nodes that connect to target node
    For Part 2, return all keys that are connected to target
    """
    keys=[]
    for key, value in dictionary.iteritems():
        if check_connected(dictionary,key,target):
            keys.append(key)
    return keys

In [16]:
count_nodes2(dictionary,0)[:5]

[0, 22, 25, 35, 41]

In [63]:
def count_groups(dictionary,keys,groups=0):
    """Count the number of groups. 
    A group is collection of programs that are not connected to each other.
    """
    
    remaining_keys=list(keys)
    
    while len(remaining_keys)>0:
        target=remaining_keys[0]
        sub_group=count_nodes2(dictionary,target)
        remaining_keys=list(set(remaining_keys)-set(sub_group)) #set subtraction - To subtract 1 list from another
        groups+=1
        print len(remaining_keys),groups
        
    
    return groups

In [64]:
count_groups(dictionary,keys)

1848 1
1808 2
1675 3
1658 4
1652 5
1640 6
1572 7
1561 8
1559 9
1482 10
1414 11
1393 12
1362 13
1228 14
1144 15
924 16
832 17
831 18
819 19
818 20
816 21
812 22
810 23
802 24
761 25
754 26
753 27
749 28
748 29
745 30
736 31
730 32
719 33
701 34
661 35
660 36
656 37
655 38
654 39
652 40
633 41
595 42
565 43
545 44
541 45
539 46
521 47
515 48
514 49
510 50
503 51
502 52
492 53
490 54
483 55
430 56
418 57
412 58
411 59
406 60
403 61
402 62
399 63
398 64
376 65
372 66
364 67
353 68
341 69
340 70
339 71
338 72
335 73
327 74
324 75
316 76
306 77
288 78
278 79
267 80
266 81
265 82
264 83
253 84
246 85
243 86
239 87
237 88
224 89
223 90
220 91
219 92
210 93
206 94
195 95
188 96
183 97
179 98
177 99
174 100
169 101
164 102
163 103
162 104
160 105
158 106
157 107
156 108
147 109
146 110
145 111
144 112
140 113
138 114
134 115
127 116
125 117
121 118
120 119
118 120
116 121
110 122
109 123
108 124
105 125
104 126
103 127
101 128
98 129
97 130
95 131
94 132
89 133
84 134
83 135
81 136
80 137
76 138

186